# Live Digit Recognition App using Tkinter and Pillow
### Author: HASEEB Sheikh Muhammad
### Date: 14/06/2025

This is my first ever machine learning project, it was quite simple to just do ML and stop, so I decided to add a GUI to it, but to make it even more challenging, I made it live, this digit recogntion program will give live predictions as you draw (only if you pause drawing, to save resources)


This is a digit recognition program using Tensorflow in Python  
I will take it step-by-step:
1. Import libraries and dataset
2. Verify dataset is good
3. Build the model
4. Train the model
5. Test the model

For GUI:  
1. Design the GUI in Paint or other software
2. Create the widgets in Tkinter
3. Add functionality to the widgets
4. Load the saved model
5. Preprocess user input
6. Make predictions  


In [133]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Input

In [122]:
# Load the MNIST dataset
numbers = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = numbers.load_data()

In [123]:
#Normalize the data
x_train = x_train / 255.0
x_test = x_test / 255.0

In [124]:
# Define the model
model = Sequential([
    Input((28,28,1)),
    Flatten(),
    Dense(128, activation = 'relu'),
    Dense(128, activation = 'relu'),
    Dense(64, activation = 'relu'),
    Dense(10, activation = 'softmax')
])
# Compile the model
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])


In [125]:
# Train the model
model.fit(x_train, y_train, epochs=20)

Epoch 1/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.8733 - loss: 0.4176
Epoch 2/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9684 - loss: 0.1034
Epoch 3/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9782 - loss: 0.0685
Epoch 4/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9831 - loss: 0.0540
Epoch 5/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9867 - loss: 0.0415
Epoch 6/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9890 - loss: 0.0333
Epoch 7/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9898 - loss: 0.0320
Epoch 8/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9915 - loss: 0.0260
Epoch 9/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9919 - loss: 0.0242
Epoch 10/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.9925 - loss: 0.0225
Epoch 11/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9943 - loss: 0.0184
Epoch 12/20
1875/1875 ━━━━━━━━

In [126]:
# Test the model
test_loss, test_acc = model.evaluate(x_test, y_test)
# Print the results
print(f'Test Loss:{test_loss}, Test accuracy: {test_acc}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9738 - loss: 0.1346  
Test Loss:0.1154850572347641, Test accuracy: 0.9793000221252441


In [127]:
# Save the model
model.save('mnistdigit_model.h5')

# Building the App

Even though the model has been trained and tested, I want to make an app to make use of this model to detect the digits live and better visualize the output from the softmax layer. All of this will be done using Tkinter and Pillow. Even though the code below is part of one program, I am going to import the required libraries here instead of top.

In [128]:
# Import necessary libraries for GUI
import tkinter as tk
from tkinter import Canvas, Button, Label
from PIL import Image, ImageDraw, ImageOps
import os

In [131]:
# Define the application as a class
class GUIApp:
    def __init__(self, master):
        self.master = master
        master.title("Digit Recognizer") # Give title to the tkinter window
        master.geometry("1280x720") # Set the size of the window
        master.resizable(True, True) # Make the window resizable
        master.configure(bg="#0f0606") # Set the background color of the window

        ''' So, that is the basic setup for tkinter, we are missing the defined functions which will be defined below,
        however I wanted to add an important note here about what I am about to do now, I am going to be using Pillow
        to create a Pillow image which will basically act as invisible layer to extract the data from the user input,
        in this case, the handrawn digit, the reason why I am using both Tkinter and Pillow is that Tkinter will not
        give me the pixel data from the user input, but Pillow will, so Tkinter is mainly used here to visualize the
        input and provide the GUI, whereas Pillow is being used to extract the user input "invisibly" and feed the
        data to the machine learning model.'''

        self.image = Image.new('L',(280,280)) # Create the Pillow Image, it is grayscale 'L' , 280 x 280 pixels
        self.pillowImage = ImageDraw.Draw(self.image) # Ready the image to be drawn on, named as such for clarity

        self.last_x, self.last_y = None, None # Store coordinated from last position of drawing lines
        self.drawing = False # Flag to know whether mouse is clicked or not
        self.brush_size = 18  # Set the brush size for drawing

        self.predict_id = None 
        ''' This stores the ID number from the after() call of Tkinter, to check for scheduled
        predictions, and cancel the previous ones, this is necessary to reduce the number of prediction calls while
        also making the app live and responsive without wasting many resources. '''

        self.setup_model() # Setup the trained model to recognize the new input
        self.create_widgets() # Create Widgets on the app   
        self.reset_canvas() # Reset the whole canvas


    # Define the model setup
    def setup_model(self):
        self.model = keras.models.load_model('mnistdigit_model.h5')
        self.model_status = "Model Successfully Loaded"

    def create_widgets(self):
        # This function creates the widgets for the GUI
            
        # This is the main frame for the whole program
        main_frame = tk.Frame(self.master, bg="#0f0606", bd=0, relief="flat", padx=30, pady=30, highlightbackground="#2f0000", highlightthickness=2)
        main_frame.pack(padx=20, pady=20, fill="both", expand=True)
        main_frame.grid_columnconfigure((0,1), weight=1)
        main_frame.grid_rowconfigure(0, weight=1)

        # This is the smaller canvas for the user input
        canvas_frame = tk.Frame(main_frame, bg= "#0f0606")
        canvas_frame.grid(row=0, column=0, padx=20, pady=20, sticky="nsew")
        canvas_frame.grid_rowconfigure((0,1,3), weight=0)
        canvas_frame.grid_rowconfigure(2, weight=1)

        # This is the the instruction label
        instruction_label = Label(canvas_frame, text="Draw a digit (0-9) in the box below", font=("Helvetica", 16, "bold"), bg="#200b0b", fg="white")
        instruction_label.pack(pady=10)
        # This is the canvas where the user will draw the digit
        self.canvas = Canvas(canvas_frame, width=280, height=280, bg="black", cursor="crosshair", bd = 2, relief="ridge", highlightbackground="#200b0b", highlightthickness=2)
        self.canvas.pack(pady=10)

        # Adding drawing binding to the canvas the actual functions will be defined below
        self.canvas.bind("<Button-1>", self.start_draw)
        self.canvas.bind("<B1-Motion>", self.draw_line)
        self.canvas.bind("<ButtonRelease-1>", self.stop_draw)

        # The clear button to clear the canvas
        clear_button = Button(canvas_frame, text="Clear Canvas", command = self.reset_canvas, font=("Helvetica", 14), bg="#200b0b", fg="white", activebackground="#2f0000", activeforeground="white")
        clear_button.pack(pady=10)
        
        # The model status label will be place here, the status will be updated after the model is loaded
        self.success_label = Label(canvas_frame, text=self.model_status, font=("Helvetica", 12), bg="#0f0606", fg="white")
        self.success_label.pack(pady=10)

        # Now definiing the result area where the prediction will be shown
        result_frame = tk.Frame(main_frame, bg="#0f0606")
        result_frame.grid(row=0, column=1, padx=20, pady=20, sticky="nsew") 
        result_frame.grid_rowconfigure((0,1), weight=0)
        result_frame.grid_rowconfigure(2, weight=1)
        result_frame.grid_columnconfigure(0, weight=1)

        # The result label to show the prediction text
        self.result_label = Label(result_frame, text="Prediction: ", font=("Helvetica", 16), bg="#0f0606", fg="white")
        self.result_label.pack(pady=10)

        # Shwowing the actual prediction number ? is the placeholder
        self.prediction_number = Label(result_frame, text="?", font=("Helvetica", 64, "bold"), bg="#0f0606", fg="white")
        self.prediction_number.pack(pady=10)

        # Prediction Matrix label
        prediction_matrix_label = Label(result_frame, text="Prediction Matrix:", font=("Helvetica", 12), bg="#0f0606", fg="white")
        prediction_matrix_label.pack(pady=10)

        # The actual prediction matrix to show the probabilities of each digit
        self.prediction_matrix = tk.Frame(result_frame, bg="#200b0b", bd = 1, relief = "groove") 
        self.prediction_matrix.pack(fill = "x", expand = True, padx=10, pady=10)
        
        # Create the matrix labels for each digit
        # Using a dictionary to store the labels for easy access
        self.probability_labels = {}

        # Using a loop to create labels for each digit
        for i in range(10):
            # Create a frame for each row in the prediction matrix
            row_frame = tk.Frame(self.prediction_matrix, bg="#200b0b")
            row_frame.pack(fill="x", pady = 2)
            # Create labels for the digit and its probability
            digit_label = Label(row_frame, text =f"Digit {i}:", font=("Helvetica", 12), bg="#200b0b", fg="white")
            digit_label.pack(side="left", padx=5)
            probability_label = Label(row_frame, text="0.00%", font=("Helvetica", 12), bg="#200b0b", fg="white")
            probability_label.pack(side="right", padx=5)
            # Store the probability label in the dictionary
            self.probability_labels[i] = {"frame" : row_frame, "probability": probability_label}


    # Define the functions for the GUI actions
    # This function defines the actions when the mouse button is pressed to start drawing
    def start_draw(self,event):
        """ This function is called when the mouse button is pressed on the canvas. Records the starting position
        of the drawing and sets the drawing flag to True. """
        self.drawing = True
        self.last_x, self.last_y = event.x, event.y
        # Drawing on the canvas for visualization
        self.canvas.create_oval(event.x - self.brush_size/2, event.y - self.brush_size/2, event.x + self.brush_size/2, event.y + self.brush_size/2, fill="white", outline="white")
        # Drawing on the Pillow image for data extraction
        self.pillowImage.ellipse([event.x - self.brush_size/2, event.y - self.brush_size/2, event.x + self.brush_size/2, event.y + self.brush_size/2], fill="white")
        # Schedule the prediction, function defined below
        self.schedule_prediction()

    # This function defines the actions when the mouse is moved while the button is pressed to draw lines
    def draw_line(self, event):
        if self.drawing:
            self.canvas.create_line(self.last_x, self.last_y, event.x, event.y, fill="white", width=self.brush_size, capstyle=tk.ROUND, smooth=True)    
            # Drawing on the Pillow image for data extraction
            self.pillowImage.line([self.last_x, self.last_y, event.x, event.y], fill="white", width = self.brush_size, joint ="curve")
            self.last_x, self.last_y = event.x, event.y
            self.schedule_prediction()  # Reschedule the prediction

    # This function defines the actions when the mouse button is released to stop drawing
    def stop_draw(self, event):
        self.drawing = False
        self.predict_digit() # Predict the digit when the mouse button is released
        self.cancel_prediction() # Cancel any scheduled predictions

    # Clear the canvas upon clicking the clear button and reset the Pillow image
    def reset_canvas(self):
        self.canvas.delete("all")  # Clear the canvas
        self.image.paste(0, (0, 0, 280, 280)) # Create a new Pillow Image
        self.pillowImage = ImageDraw.Draw(self.image)
        # Reset prediction, and prediction matrix
        self.prediction_number.config(text="?")
        for i in range(10):
            self.probability_labels[i]["probability"].config(text="0.00%")

    def schedule_prediction(self):
        if self.predict_id:
            self.master.after_cancel(self.predict_id) # Cancel the prediction if it is already scheduled
        self.predict_id = self.master.after(500, self.predict_digit) # Create a new prediction after 500ms

    def cancel_prediction(self):
        if self.predict_id:
            self.master.after_cancel(self.predict_id)
            self.predict_id = None # Reset the predict_id to None

    # Ready the pillow image for prediction
    def preprocess_image(self):
        # Reize the image to 28x28 pixels
        img = self.image.resize((28, 28), Image.LANCZOS)
        # Since the model was trained on a black background with white digits, we need to invert the colors for the prediction
        # Convert the image to a numpy array and normalize it
        img_array = np.array(img).astype('float32') / 255.0
        # Reshape the array to match the input shape of the model
        final_array = img_array.reshape(1, 28, 28,1)  # Add batch dimension
        return final_array

    def predict_digit(self):
        # Use the preprocess_image function to get the image ready for prediction
        processed_image = self.preprocess_image()
        # Make the prediction using the model
        predictions = self.model.predict(processed_image, verbose=0) # verbose=0 to suppress output
        probabilities = predictions[0]
        predicted_digit = np.argmax(probabilities)
        # Update the prediction number label
        self.prediction_number.config(text=str(predicted_digit))
        # Update the probability matrix
        for i in range(10):
            self.probability_labels[i]["probability"].config(text=f"{100 * probabilities[i]:.2f}%")

In [132]:
window = tk.Tk()
app = GUIApp(window)
window.mainloop()

# That is all!
What a rewarding experience, so much learning done in such short amount of time  
I want to write down some of my takeaways from this project:  
1. Machine learning basics (layers etc.).
2. Importance of input shape, this was the biggest 'bug' for me, since I did not really understand the shapes of the training data, and was doing the preprocessing wrong.
3. Tkinter GUI in somewhat depth, (windows, layouts, widgets, parenting, canvas, binding, events, etc).
4. Importance of planning out the project first, visualizing the project before starting.